In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
print(os.listdir("../input"))

# Any results you write to the current directory are saved as output.

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
import cv2
import random
import tensorflow as tf
from tensorflow import keras
from sklearn.model_selection import train_test_split
from keras.utils import to_categorical
from keras.models import Sequential
from keras.layers.core import Dense, Dropout, Flatten, Activation
from keras.layers import Conv2D, MaxPooling2D
from keras.utils import np_utils
from keras.layers.normalization import BatchNormalization
from keras.callbacks import EarlyStopping

%matplotlib inline

In [ ]:
directory = '../input/train'
test_directory = '../input/test/'
random_test = '../input/driver/'

classes = ['c0','c1','c2','c3','c4','c5','c6','c7','c8','c9']

In [ ]:
# Defining the size of the image
img_size1 = 240
img_size2 = 240

In [ ]:
# Train class image for display

for i in classes:
    path = os.path.join(directory, i)
    for img in os.listdir(path):
        img_array = cv2.imread(os.path.join(path,img), 0)
        plt.imshow(img_array, cmap="gray")
        plt.show()
        break
    break

In [ ]:
# Test class image for display.

test_array = []

for img in os.listdir(test_directory):
    img_array = cv2.imread(os.path.join(test_directory, img), 0)
    test_array = img_array
    plt.imshow(img_array, cmap='gray')
    plt.show()
    break

In [ ]:
img_array.shape

In [ ]:
training_data = []
i = 0

def create_training_data():
    for category in classes:
        path = os.path.join(directory, category)
        class_num = classes.index(category)
        
        for img in os.listdir(path):
            img_array = cv2.imread(os.path.join(path, img), 0)
            new_img = cv2.resize(img_array, (img_size1, img_size2))
            training_data.append([new_img, class_num])

In [ ]:
testing_data = []
i=0

def create_test_data():
    for img in os.listdir(test_directory):
        path = os.path.join(test_directory, img)
        img_array = cv2.imread(path, 0)
        new_img = cv2.resize(img_array, (img_size1, img_size2))
        testing_data.append([img, new_img])

In [ ]:
create_test_data()

In [ ]:
create_training_data()

In [ ]:
print(len(training_data))
print(len(testing_data))

In [ ]:
random.shuffle(training_data)

In [ ]:
x = []
y = []

In [ ]:
for features, label in training_data:
    x.append(features)
    y.append(label)

In [ ]:
x[1].shape

In [ ]:
len(x)

In [ ]:
len(y)

In [ ]:
X = np.array(x).reshape(-1, img_size2, img_size1, 1)
X[0].shape

In [ ]:
x_train,x_test,y_train,y_test = train_test_split(X,y,test_size=0.3,random_state=50)

In [ ]:
Y_train = np_utils.to_categorical(y_train,num_classes=10)
Y_test = np_utils.to_categorical(y_test,num_classes=10)

In [ ]:
model = Sequential()

In [ ]:
model.add(Conv2D(32, kernel_size=(3,3), activation = 'relu', input_shape=(240,240,1)))
model.add(BatchNormalization())


In [ ]:
model.add(Conv2D(32,kernel_size=(3,3),activation='relu',input_shape=(240,240,1)))
model.add(BatchNormalization())
model.add(Conv2D(32,kernel_size=(3,3),activation='relu',padding='same'))
model.add(BatchNormalization(axis = 3))
model.add(MaxPooling2D(pool_size=(2,2),padding='same'))
model.add(Dropout(0.3))

In [ ]:
model.add(Conv2D(64,kernel_size=(3,3),activation='relu',padding='same'))
model.add(BatchNormalization())
model.add(Conv2D(64,kernel_size=(3,3),activation='relu',padding='same'))
model.add(BatchNormalization(axis = 3))
model.add(MaxPooling2D(pool_size=(2,2),padding='same'))
model.add(Dropout(0.3))

In [ ]:
model.add(Conv2D(128,kernel_size=(3,3),activation='relu',padding='same'))
model.add(BatchNormalization())
model.add(Conv2D(128,kernel_size=(3,3),activation='relu',padding='same'))
model.add(BatchNormalization(axis = 3))
model.add(MaxPooling2D(pool_size=(2,2),padding='same'))
model.add(Dropout(0.5))

In [ ]:
model.add(Flatten())
model.add(Dense(units = 512,activation='relu'))
model.add(BatchNormalization())
model.add(Dropout(0.5))
model.add(Dense(units = 128,activation='relu'))
model.add(Dropout(0.25))
model.add(Dense(10,activation='softmax'))

In [ ]:
model.summary()

In [ ]:
model.compile(loss='categorical_crossentropy',metrics=['accuracy'],optimizer='adam')

In [ ]:
callbacks = [EarlyStopping(monitor='val_acc',patience=5)]

In [ ]:
batch_size = 50
n_epochs = 20

In [ ]:
results = model.fit(x_train,Y_train,batch_size=batch_size,epochs=n_epochs,verbose=1,validation_data=(x_test,Y_test),callbacks=callbacks)

In [ ]:
# Plot training & validation accuracy values
plt.plot(results.history['acc'])
plt.plot(results.history['val_acc'])
plt.title('Model accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['Train', 'Test'], loc='upper left')
plt.show()

# Plot training & validation loss values
plt.plot(results.history['loss'])
plt.plot(results.history['val_loss'])
plt.title('Model loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['Train', 'Test'], loc='upper left')
plt.show()

In [ ]:
preds = model.predict(np.array(testing_data[0][1]).reshape(-1,img_size2,img_size1,1))

In [ ]:
model.save_weights('./driverdistraction_lr_weights.h5', overwrite=True)

In [ ]:
model.save('./driverdistraction_lr_weights.h5')

In [ ]:
loaded_model = load_model('../input/driver-distraction/driverdistraction_lr_weights.h5')

In [ ]:
test_data = np.array(testing_data[3000][1]).reshape(-1,img_size2,img_size1,1)

In [ ]:
preds = loaded_model.predict(test_data)
preds

In [ ]:
print('Predicted: {}'.format(np.argmax(preds)))
new_img = cv2.resize(testing_data[3000][1],(img_size2,img_size1))
plt.imshow(new_img,cmap='gray')
plt.show()